In [1]:
from genericpath import exists
from lib2to3.pytree import convert
from pickle import NONE
import pandas as pd
from glob import glob
import csv
import pandas as pd
from pandasql import sqldf
import json
import re
from datetime import datetime
import os
import pandas as pd
from pandasql import sqldf
import csv
import math
from haversine import haversine

In [2]:
# import suburbs, only get those within 100km from cbd
df = pd.read_csv ('australian_suburbs_1.csv')
df = sqldf('''

    SELECT suburb
         , state
         , postcode
      FROM df
     WHERE state = 'NSW'
       AND lat BETWEEN (-33.8708-0.9) AND (-33.8708+0.9)
       AND lng BETWEEN (151.2073-0.9) AND (151.2073+0.9)

''')
df_dict = df.to_dict('records')


suburbs = []
for record in df_dict:
    suburb = record['suburb'].replace(' ', '-')
    state = 'nsw'
    postcode = record['postcode']
    suburbs.append(f'{suburb}-{state}-{postcode}')

In [3]:
import csv
def read_csv(csv_file_name: str):
    with open(csv_file_name, 'r') as f:
        reader = csv.DictReader(f)
        return [row for row in reader]
stations = read_csv(r"C:/Users/tyler/DA_Tyler/geo_data/Sydney_StationEntrances2020_v4.csv")

In [4]:
def clean_station_data(raw_data: dict) -> dict:
    cleaned_station_data = raw_data.copy()
    lat = cleaned_station_data.get('LAT')
    lng = cleaned_station_data.get('LNG')
    if lat:
        cleaned_station_data['LAT'] = float(lat)
    if lng:
        cleaned_station_data['LNG'] = float(lng)
    return cleaned_station_data

In [5]:
cleaned_stations = map(clean_station_data, stations)
cleaned_stations = [{'lat':s['LAT'],"lng":s['LNG']} for s in cleaned_stations ]


In [6]:
#take two location, calculate distance
def station_distance(cleaned_station: dict, lat,lng: dict):
    s1 = (cleaned_station['lat'], cleaned_station['lng'])
    s2 = (lat, lng)
    return haversine(s1, s2)

In [7]:
#return stations within x kms
def within_km(lat,lng, distance):
    lng_diff = float(distance /110.0) 
    lat_diff = float(distance /110.0)
    return [s for s in cleaned_stations if  (s['lng'] - lng_diff) <= lng <= (s['lng'] + lng_diff) and  (s['lat'] - lat_diff) <= lat <= (s['lat'] + lat_diff)]
     

In [8]:
# return the distance to the closest station.
def nearby_station(lat,lng):
    distance = 0.5
    while distance < 100:
        if len(within_km(lat,lng, distance)) >= 1:
            stations =  [station_distance(s, lat,lng) for s in within_km(lat,lng, distance)]
            return min(stations)
        else:
            distance += 0.5

In [9]:
def read_file(file_name: str) -> str:
    with open(file_name, "r") as f:
        return f.read()

def read_json_file(file_name: str):
    return json.loads(read_file(file_name))

def combine_csv(suburbs: list):
        with open(r"C:/Users/tyler/DA_Tyler/download/www.source.com/all_csv/sydney_raw_data.csv", "w", newline="", encoding='utf-8') as f:
            field_names = ['id','type','pricefromapm','price','landsize','price_square','sold_date','auction_or_private','suburb','postcode','street_address', 'beds','baths','parking','agency','lat','lng','to_cbd', 'to_train']
            writer = csv.DictWriter(f, fieldnames = field_names)
            writer.writeheader()
            n = 0
            for suburb in suburbs:
                path = fr"C:/Users/tyler/DA_Tyler/download/www.source.com/{suburb}"
                isexist = os.path.exists(path)
                if isexist == False:
                    print(f'{suburb} is not download yet')
                else:    
                    for json_file_name in glob(fr"C:/Users/tyler/DA_Tyler/download/www.source.com/{suburb}/page_*.json"):
                        data = read_json_file(json_file_name)['props']['listingsMap']
                        for record in data:
                            id = record
                            type = data[f'{record}']['listingModel']['features']['propertyType']
                            price = data[f'{record}']['listingModel']['price']
                            price = re.findall('\$([\d,]+)',f'{price}')
                            price = int(price[0].replace(',',''))
                            if data[f'{record}']['listingModel'].get('priceFromApm') is None:
                                pricefromapm = None
                            else:    
                                pricefromapm = data[f'{record}']['listingModel']['priceFromApm']
                            landsize = data[f'{record}']['listingModel']['features']['landSize']
                            if landsize < 27:
                                landsize = 0
                            else:
                                landsize = landsize
                            if landsize >10:
                                price_square = round(price/landsize,0)
                            else:
                                price_square = 0
                            adver = data[f'{record}']['listingModel']['tags']['tagText']
                            adver_list = re.findall('(private treaty|auction|prior to auction) (\d.+)',f'{adver}')
                            if len(adver_list)==1:
                                sold_date = datetime.strptime(adver_list[0][1], "%d %b %Y").date()
                            else:
                                sold_date = None
                            if len(adver_list)==1:
                                auction_or_private = adver_list[0][0]
                            else:
                                auction_or_private = None
                            
                            suburb = data[f'{record}']['listingModel']['address']['suburb']
                            postcode = data[f'{record}']['listingModel']['address']['postcode']
                            street_address = data[f'{record}']['listingModel']['address']['street']

                            if data[f'{record}']['listingModel']['features'].get('beds') is None:
                                beds = 0
                            else:
                                beds = data[f'{record}']['listingModel']['features']['beds']

                            if data[f'{record}']['listingModel']['features'].get('baths') is None:
                                baths = 0
                            else:
                                baths = data[f'{record}']['listingModel']['features']['baths']
                        
                            if data[f'{record}']['listingModel']['features'].get('parking') is None:
                                parking = 0
                            else:
                                parking = data[f'{record}']['listingModel']['features']['parking']
                            agency = data[f'{record}']['listingModel']['branding']['agentNames']
                            if data[f'{record}']['listingModel']['address'].get('lat') is None:
                                lat = None
                            else:
                                lat = data[f'{record}']['listingModel']['address']['lat']
                            if data[f'{record}']['listingModel']['address'].get('lng') is None:
                                lng = None
                            else:    
                                lng = data[f'{record}']['listingModel']['address']['lng']
                            if lat is None or lng is None:
                                to_cnd = None
                            else:
                                to_cbd = math.sqrt((lat+33.8708)**2 + (lng-151.2073)**2)*110
                            if lat is None or lng is None:
                                to_train = None
                            else:
                                to_train = nearby_station(lat, lng)
                            writer.writerow(dict(zip(field_names, [id, type, pricefromapm, price,landsize,price_square,sold_date,auction_or_private,suburb,postcode,street_address, beds, baths,parking,agency,lat,lng,to_cbd,to_train])))
                print(f'Got {suburb}')

In [10]:
combine_csv(suburbs)

Got CROYDON PARK
Got CROYDON
Got ASHFIELD
Got HABERFIELD
Got SUMMER HILL
Got HURLSTONE PARK
Got ASHBURY
Got NEWINGTON
Got LIDCOMBE
Got BERALA
Got AUBURN
Got SYDNEY OLYMPIC PARK
Got SILVERWATER
Got REGENTS PARK
Got WENTWORTH POINT
Rookwood-nsw-2141 is not download yet
Got Rookwood-nsw-2141
Got HORNSBY
Got CONDELL PARK
Got BANKSTOWN
Got PUNCHBOWL
Got MOUNT LEWIS
Got GREENACRE
Got YAGOONA
Got CHULLORA
Got GEORGES HALL
Got VILLAWOOD
Got BASS HILL
Got CHESTER HILL
Got SEFTON
Got REVESBY
Got LANSDOWNE
Got POTTS HILL
Got BIRRONG
Bankstown-Aerodrome-nsw-2200 is not download yet
Got Bankstown-Aerodrome-nsw-2200
Got MILPERRA
Got EAST HILLS
Got PANANIA
Got PICNIC POINT
Got REVESBY HEIGHTS
Got PADSTOW
Got PADSTOW HEIGHTS
Got BELLA VISTA
Got BAULKHAM HILLS
Got CASTLE HILL
Got KELLYVILLE
Got WEST PENNANT HILLS
Got WINSTON HILLS
Got GLENHAVEN
Rouse-Hill-nsw-2155 is not download yet
Got Rouse-Hill-nsw-2155
Beaumont-Hills-nsw-2155 is not download yet
Got Beaumont-Hills-nsw-2155
Kenthurst-nsw-2156 is no